In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
df=pd.read_csv('../Data/HouseImformation.csv')

In [3]:
df['Area'] = df['Area'].str.replace(',', '').astype(float)

df=df.dropna()

isoOuter=IsolationForest(contamination=0.01)
df['OuterData']=isoOuter.fit_predict(df[["Area", "Price"]])

df=df.drop(index=df[df['OuterData'] == -1].index)

MeanPriceAddressOuter=df.groupby("Address")["Price"].mean()
df["MeanPriceA"] = df["Address"].map(MeanPriceAddressOuter)

In [10]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
x_Data = scaler.fit_transform(df[['Area','Room','Parking','Warehouse','Elevator','MeanPriceA']])
y_Data=df[['Price']].values


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold ,cross_val_score

model=Lasso(alpha=1.0)

cvM=RepeatedKFold(n_splits=10,n_repeats=3,random_state=1)

score=cross_val_score(model,x_Data,y_Data,cv=cvM)

print("Mean: ",np.mean(score))
print("Standard deviation: ", np.std(score))

Mean:  0.7468603862508736
Standard deviation:  0.038843599375816606


In [26]:
mask= np.random.rand(len(df)) < 0.8

train=df[mask]
test=df[~mask]

x_train=np.asanyarray(train[['Area','Room','Parking','Warehouse','Elevator','MeanPriceA']])
y_train=np.asanyarray(train[['Price']])

x_test=np.asanyarray(test[['Area','Room','Parking','Warehouse','Elevator','MeanPriceA']])
y_test=np.asanyarray(test[['Price']])


scaler=StandardScaler()

x_train_scaler=scaler.fit_transform(x_train)

x_test_scaler=scaler.fit_transform(x_test)


In [27]:
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

from sklearn.metrics import r2_score
print(f"Test R2 Score: {r2_score(y_test, y_pred):.2f}")

Test R2 Score: 0.75


In [18]:
joblib.dump(model, '../Models/LassoRegression.pkl')

['../Models/LassoRegression.pkl']

In [32]:
pipeline=Pipeline([
    ('scale', StandardScaler()),
    ('poly',PolynomialFeatures(degree=2)),
    ('lasso',Lasso(alpha=1.0))
])

cvMP=RepeatedKFold(n_splits=10,n_repeats=3,random_state=1)
scoreP=cross_val_score(pipeline,x_Data,y_Data,cv=cvMP)

print("Mean: ",np.mean(score))
print("Standard deviation: ", np.std(score))

c:\Users\Abolfazl\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.571e+21, tolerance: 1.283e+19
  model = cd_fast.enet_coordinate_descent(
c:\Users\Abolfazl\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.679e+21, tolerance: 1.290e+19
  model = cd_fast.enet_coordinate_descent(
c:\Users\Abolfazl\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.558e+21,

Mean:  0.7468603862508736
Standard deviation:  0.038843599375816606


c:\Users\Abolfazl\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.537e+21, tolerance: 1.214e+19
  model = cd_fast.enet_coordinate_descent(
